In [2]:
import numpy as np
import qutip as qt

In [3]:
def reconstruct_gksl_qutip(H, kraus_ops, dt):
    """
    Reconstruct the GKSL (Lindblad) generator from small-time Kraus operators.

    Parameters
    ----------
    H : qutip.Qobj
        Known physical Hamiltonian (Hermitian).
    kraus_ops : list[qutip.Qobj]
        Kraus operators at small time dt.
    dt : float
        Small time step.

    Returns
    -------
    result : dict
        {
            'H'      : Hermitian Hamiltonian,
            'L_list' : list of Lindblad jump operators (Qobj),
            'Heff'   : effective non-Hermitian Hamiltonian,
            'L_super': QuTiP superoperator (Qobj)
        }
    """

    # Convert to numpy matrices for analysis
    mats = [K.full() for K in kraus_ops]
    d = mats[0].shape[0]

    # 1) Identify the "no-jump" Kraus (largest singular value)
    norms = [np.linalg.norm(K, 2) for K in mats]
    K0 = mats[np.argmax(norms)]
    jump_list = [mats[i] for i in range(len(mats)) if i != np.argmax(norms)]

    # 2) Extract jump operators: K_j ≈ sqrt(dt) L_j
    L_list = [qt.Qobj(K / np.sqrt(dt)) for K in jump_list]

    # 3) Estimate Heff = H - i/2 Σ L†L
    Sigma = sum([L.dag() * L for L in L_list], qt.Qobj(np.zeros_like(H.full())))
    Heff = H - 0.5j * Sigma

    # 4) Construct the GKSL (Lindbladian) superoperator
    L_super = qt.liouvillian(H, L_list)

    return {
        'H': H,
        'L_list': L_list,
        'Heff': Heff,
        'L_super': L_super
    }

In [4]:
# Define a 2-level system: H = (ω/2) σ_z, amplitude damping with L = √γ |0><1|
omega = 1.5
gamma = 0.3

sz = qt.sigmaz()
sm = qt.sigmam()

H_true = 0.5 * omega * sz
L_true = np.sqrt(gamma) * sm

# Small time step
dt = 1e-3

# Construct Kraus operators at small dt
K0 = qt.qeye(2) - 1j * dt * H_true - 0.5 * dt * (L_true.dag() * L_true)
K1 = np.sqrt(dt) * L_true

kraus_ops = [K0, K1]

# Reconstruct GKSL form
res = reconstruct_gksl_qutip(H_true, kraus_ops, dt)

In [11]:
print("=== Reconstructed GKSL ===")
print("H (Hermitian):")
print(res['H'])
print("\nEffective Heff (non-Hermitian):")
print(res['Heff'])
print("\nLindblad jump operators:")
for i, L in enumerate(res['L_list']):
    print(f"L{i} =\n{L}")
print("\nLindbladian superoperator (matrix form):")
print(res['L_super'].full())

# Sanity check: evolve a state using both Kraus and GKSL
psi0 = qt.basis(2, 1)        # excited state |1>
rho0 = psi0 * psi0.dag()

# 1) Evolution via Kraus operators
rho_kraus = sum(K * rho0 * K.dag() for K in kraus_ops)

# 2) Evolution via GKSL to first order:
# Vectorize rho -> apply L_super -> unvectorize
rho_vec = qt.operator_to_vector(rho0)
rho_vec_next = rho_vec + dt * (res['L_super'] * rho_vec)
rho_gksl = qt.vector_to_operator(rho_vec_next)

# 3) Compare
print("\nTrace distance between Kraus and GKSL step:")
print((rho_kraus - rho_gksl).norm())

# Optional: display both
print("\nρ (Kraus step):\n", rho_kraus.full())
print("\nρ (GKSL step):\n", rho_gksl.full())

=== Reconstructed GKSL ===
H (Hermitian):
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ 0.75  0.  ]
 [ 0.   -0.75]]

Effective Heff (non-Hermitian):
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.75-0.15j  0.  +0.j  ]
 [ 0.  +0.j   -0.75+0.j  ]]

Lindblad jump operators:
L0 =
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[0.         0.        ]
 [0.54772256 0.        ]]

Lindbladian superoperator (matrix form):
[[-0.3 +0.j   0.  +0.j   0.  +0.j   0.  +0.j ]
 [ 0.  +0.j  -0.15+1.5j  0.  +0.j   0.  +0.j ]
 [ 0.  +0.j   0.  +0.j  -0.15-1.5j  0.  +0.j ]
 [ 0.3 +0.j   0.  +0.j   0.  +0.j   0.  +0.j ]]

Trace distance between Kraus and GKSL step:
5.624999999120917e-07

ρ (Kraus step):
 [[0.        +0.j 0.        +0.j]
 [0.        +0.j 1.00000056+0.j]]

ρ (GKSL step):
 [[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


# Quantum Speed Limit

In [12]:
def bures_angle(rho0: qt.Qobj, rhot: qt.Qobj) -> float:
    F = qt.metrics.fidelity(rho0, rhot)  # Uhlmann fidelity
    F = min(max(F, 0.0), 1.0)            # clamp num. noise
    return np.arccos(np.sqrt(F))

In [13]:
def lindbladian_action_from_super(L_super: qt.Qobj, rho: qt.Qobj) -> qt.Qobj:
    """Apply superoperator to rho (vectorize, multiply, unvectorize)."""
    vec = qt.operator_to_vector(rho)
    vec_out = L_super * vec
    return qt.vector_to_operator(vec_out)

In [14]:
def schatten_norm(A: qt.Qobj, p: float) -> float:
    """Schatten p-norm of operator A (as matrix on Hilbert space)."""
    s = np.linalg.svd(A.full(), compute_uv=False)
    if p == 1:
        return float(np.sum(s))
    elif p == 2:
        return float(np.linalg.norm(s))  # Frobenius equals sqrt(sum s^2)
    elif p == np.inf:
        return float(np.max(s))
    else:
        # generic p > 0
        return float((s**p).sum()**(1.0/p))

In [15]:
def qsl_deffner_lutz(H: qt.Qobj,
                     L_list: list[qt.Qobj],
                     rho0: qt.Qobj,
                     tau: float,
                     steps: int = 1000):
    """
    Compute Deffner-Lutz QSL bounds for GKSL dynamics generated by (H, L_list).

    Returns:
      dict with angles, average norms, tau_QSL^(p) for p=1,2,inf, and pass/fail flags.
    """
    # Build superoperator and integrate trajectory
    L_super = qt.liouvillian(H, L_list)
    tlist = np.linspace(0.0, tau, steps+1)
    sol = qt.mesolve(H, rho0, tlist, c_ops=L_list, e_ops=[])
    rhos = sol.states

    # Bures angle between initial and final
    angle = bures_angle(rho0, rhos[-1])
    sin2 = np.sin(angle)**2

    # Average norms Λ_τ^{(p)} = (1/τ) ∫ ||L(ρ_t)||_p dt   (Riemann sum)
    # We evaluate L(ρ_t) at each sample and average (exclude final to match dt grid)
    L_rhos = [lindbladian_action_from_super(L_super, r) for r in rhos[:-1]]
    # time step
    dt = tau / steps

    def avg_norm(p):
        vals = [schatten_norm(A, p) for A in L_rhos]
        return (dt * np.sum(vals)) / tau

    Lambda_1 = avg_norm(1)
    Lambda_2 = avg_norm(2)
    Lambda_inf = avg_norm(np.inf)

    # Guard against division by very small numbers
    eps = 1e-14
    tau_qsl_1 = sin2 / max(Lambda_1, eps)
    tau_qsl_2 = sin2 / max(Lambda_2, eps)
    tau_qsl_inf = sin2 / max(Lambda_inf, eps)
    tau_qsl_max = max(tau_qsl_1, tau_qsl_2, tau_qsl_inf)

    results = {
        "tau": float(tau),
        "angle_bures": float(angle),
        "sin2_angle": float(sin2),
        "Lambda_1": float(Lambda_1),
        "Lambda_2": float(Lambda_2),
        "Lambda_inf": float(Lambda_inf),
        "tau_qsl_1": float(tau_qsl_1),
        "tau_qsl_2": float(tau_qsl_2),
        "tau_qsl_inf": float(tau_qsl_inf),
        "tau_qsl_max": float(tau_qsl_max),
        "pass_1": tau >= tau_qsl_1 - 1e-10,
        "pass_2": tau >= tau_qsl_2 - 1e-10,
        "pass_inf": tau >= tau_qsl_inf - 1e-10,
        "pass_overall": tau >= tau_qsl_max - 1e-10
    }
    return results

In [16]:
# ============================
# Example usage with your reconstructed GKSL (res):
# - res['H']:   Hamiltonian (Qobj, Hermitian)
# - res['L_list']: list of jump operators (Qobj)
# Provide an initial state rho0 and a target time tau.
# ============================

# If you already have res from the reconstruction step:
H_gksl = res['H']
L_ops = res['L_list']

# Example initial state (excited state for a qubit)
psi0 = qt.basis(H_gksl.shape[0], 1) if H_gksl.shape[0] >= 2 else qt.basis(H_gksl.shape[0], 0)
rho0 = psi0 * psi0.dag()

# Choose an evolution time tau to test (physical units: ħ=1 here)
tau = 0.2  # adjust as you like

# Compute and report
qsl = qsl_deffner_lutz(H_gksl, L_ops, rho0, tau, steps=2000)

print("=== Quantum Speed Limit (Deffner-Lutz) Check ===")
print(f"Target time τ:                  {qsl['tau']:.6g}")
print(f"Bures angle ℒ(ρ0, ρτ):          {qsl['angle_bures']:.6g}  (sin²ℒ = {qsl['sin2_angle']:.6g})")
print(f"Average generator norms Λ_τ^p:   ||·||₁={qsl['Lambda_1']:.6g},  ||·||₂={qsl['Lambda_2']:.6g},  ||·||_∞={qsl['Lambda_inf']:.6g}")
print(f"QSL bounds τ_QSL^(1,2,∞):        {qsl['tau_qsl_1']:.6g}, {qsl['tau_qsl_2']:.6g}, {qsl['tau_qsl_inf']:.6g}")
print(f"Max QSL bound τ_QSL^max:         {qsl['tau_qsl_max']:.6g}")
print(f"Pass p=1?  {qsl['pass_1']};  Pass p=2?  {qsl['pass_2']};  Pass p=∞?  {qsl['pass_inf']}")
print(f"PASS OVERALL? {qsl['pass_overall']}")

=== Quantum Speed Limit (Deffner-Lutz) Check ===
Target time τ:                  0.2
Bures angle ℒ(ρ0, ρτ):          0  (sin²ℒ = 0)
Average generator norms Λ_τ^p:   ||·||₁=0,  ||·||₂=0,  ||·||_∞=0
QSL bounds τ_QSL^(1,2,∞):        0, 0, 0
Max QSL bound τ_QSL^max:         0
Pass p=1?  True;  Pass p=2?  True;  Pass p=∞?  True
PASS OVERALL? True
